## Loading library and making instance off class


In [1]:
import datetime as dt
from dateutil import parser
import plotly.graph_objects as go
import pandas as pd
import sys
sys.path.append("../")

In [2]:
from api.oanda_api import OandaApi
from infrastructure.instrument_collection import InstrumentCollection as ic
from db.db import DataBaseMongo
pd.set_option('display.max_rows', None)

In [3]:
database = DataBaseMongo()
icol = ic()
api = OandaApi()

In [4]:
load_icol = icol.DB_LoadInstruments()

## Preparing the pairs


In [5]:
pairs = []
for pair, val in icol.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [6]:
pairs[0:5]

['EUR_GBP', 'GBP_NZD', 'GBP_PLN', 'GBP_CHF', 'GBP_CAD']

In [7]:
candle_data = []
for pair in pairs:
    print(pair)
    candle_temp = api.get_candles_df(
        pair_name=pair, granularity='D', count=3000)
    candle_temp['gain'] = (
        (candle_temp.mid_c - candle_temp.mid_o) / candle_temp.mid_o) * 100
    candle_temp['pair'] = pair
    if '_GBP' in pair:
        candle_temp['gain'] = candle_temp['gain'] * -1
    candle_data.append(candle_temp)
candles_df = pd.concat(candle_data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

EUR_GBP
GBP_NZD
GBP_PLN
GBP_CHF
GBP_CAD
GBP_HKD
GBP_ZAR
GBP_JPY
GBP_AUD
GBP_SGD
GBP_USD


## Querying for data from database


In [8]:
forex_calendar_data = database.query_all(DataBaseMongo.FOREX_CALENDAR)
forex_calendar_df = pd.DataFrame.from_dict(forex_calendar_data)
forex_calendar_df['date'] = pd.to_datetime(
    forex_calendar_df['date']).dt.date
forex_calendar_df_uk = forex_calendar_df[forex_calendar_df.country == 'united kingdom'].copy(
)

## Doing some quick cleaning off data


In [9]:
for column in ['actual', 'previous', 'forecast']:
    for symbol in ['$', 'B', 'K', '£', '¥', '€', '₩', '%']:
        forex_calendar_df_uk[column] = forex_calendar_df_uk[column].str.replace(
            symbol, '')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].replace(
        '', '0')
    forex_calendar_df_uk[column] = forex_calendar_df_uk[column].astype(float)

## Matching date and preparing for merge


In [10]:
date_match = {}
for original_date in forex_calendar_df_uk.date.unique():
    matched_date = original_date
    tries = 0
    while matched_date not in candles_df.time.values:
        matched_date = matched_date + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed to find a matching date.")
            break
    date_match[original_date] = matched_date

In [11]:
forex_calendar_df_uk['original_date'] = forex_calendar_df_uk['date']
forex_calendar_df_uk['date'] = [date_match[original_date]
                                for original_date in forex_calendar_df_uk['date']]
forex_calendar_df_uk['delta_previous'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['previous']
forex_calendar_df_uk['delta_forecast'] = forex_calendar_df_uk['actual'] - \
    forex_calendar_df_uk['forecast']

## Merging the data


In [12]:
candles_analyze = candles_df[['time', 'pair', 'gain']].copy()

In [13]:
merged_dataframe = pd.merge(

    left=candles_analyze, right=forex_calendar_df_uk, left_on='time', right_on='date')

In [17]:
merged_dataframe.category.unique()

array(['balance of trade', 'inflation rate', 'claimant count change',
       'consumer confidence', 'calendar', 'interest rate',
       'retail sales mom', 'composite pmi', 'monthly gdp mom',
       'monthly gdp yoy', 'manufacturing pmi', 'services pmi',
       'unemployment rate', 'gdp growth rate', 'gdp annual growth rate'],
      dtype=object)

In [ ]:
merged_catagory_dataframe = merged_dataframe[merged_dataframe.category ==
                                             'manufacturing pmi']

In [ ]:
merged_catagory_dataframe[0:5]

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_previous,delta_forecast
1,2020-01-15,EUR_GBP,0.413987,2020-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.3,1.5,1.5,2020-01-15,-0.2,-0.2
7,2020-02-19,EUR_GBP,-0.080102,2020-02-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.8,1.3,1.4,2020-02-19,0.5,0.4
13,2020-03-25,EUR_GBP,1.254124,2020-03-25,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.7,1.8,1.5,2020-03-25,-0.1,0.2
18,2020-04-22,EUR_GBP,0.540048,2020-04-22,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.5,1.7,1.3,2020-04-22,-0.2,0.2
22,2020-05-20,EUR_GBP,0.089213,2020-05-20,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,0.8,1.5,0.8,2020-05-20,-0.7,0.0


## Analyzing the data
